In [1]:
import pandas as pd 
import numpy as np
import dask.dataframe as dd

# parallelize the work using dask
from dask.distributed import Client
client = Client()

/home/gaia/.local/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40733 instead
  warnings.warn(


In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:40733/status,
Dashboard: http://127.0.0.1:40733/status,Workers: 4
Total threads: 8,Total memory: 3.74 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40083,Workers: 4
Dashboard: http://127.0.0.1:40733/status,Total threads: 8
Started: Just now,Total memory: 3.74 GiB
Comm: tcp://127.0.0.1:36435,Total threads: 2
Dashboard: http://127.0.0.1:42241/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:33241,


In [3]:
df0 = dd.read_csv(r"./demo_Z0.00014/output_*.csv", dtype={'BEvent': float})

### Wolf Rayet - Black Hole systems

In [4]:
df0_WRBH = df0.loc[( ((df0.PhaseBSE_0==8) | (df0.PhaseBSE_0==7)) & (df0.PhaseBSE_1==14.0) ) | ( ((df0.PhaseBSE_1==8) | (df0.PhaseBSE_1==7)) & (df0.PhaseBSE_0==14.0) )]
id0_WRBH = df0_WRBH['ID'].drop_duplicates().compute()

2024-05-09 12:24:27,083 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 687.58 MiB -- Worker memory limit: 0.93 GiB


2024-05-09 12:24:27,776 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle cc27df5b9602187858501a9da3161ff3 initialized by task ('shuffle-transfer-cc27df5b9602187858501a9da3161ff3', 88) executed on worker tcp://127.0.0.1:37309
2024-05-09 12:25:13,973 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle cc27df5b9602187858501a9da3161ff3 deactivated due to stimulus 'task-finished-1715250313.9714963'
2024-05-09 12:25:27,613 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle e8b9ccfcb9ab5176c1a892eee3d2c805 initialized by task ('shuffle-transfer-e8b9ccfcb9ab5176c1a892eee3d2c805', 94) executed on worker tcp://127.0.0.1:37213
2024-05-09 12:26:23,043 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle e8b9ccfcb9ab5176c1a892eee3d2c805 deactivated due to stimulus 'task-finished-1715250383.04085'
2024-05-09 12:30:36,490 - distributed.nanny - WARNING - Restarting worker
2024-05-09 12:37:38,428 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 4f0b07cb2c6b

### GW BBH merge

In [5]:
id0_GW = df0.loc[((df0.PhaseBSE_0 == 14.0) & (df0.PhaseBSE_1 == 14.0) & ((df0.GWtime + df0.BWorldtime) < int(14e+03)))]['ID'].drop_duplicates().compute()

2024-05-09 12:25:26,964 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 696.43 MiB -- Worker memory limit: 0.93 GiB
2024-05-09 12:25:45,372 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 649.86 MiB -- Worker memory limit: 0.93 GiB
2024-05-09 12:25:49,010 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

In [6]:
id0_both = np.intersect1d(id0_WRBH, id0_GW)

In [7]:
print('Binary systems of a Wolf-Rayet and a Black Hole: ', len(id0_WRBH))
print('Binary systems that merge via GW: ', len(id0_GW))
print('Binary systems that evolve from a WR-BH and merge via GW: ', len(id0_both))

Binary systems of a Wolf-Rayet and a Black Hole:  13495
Binary systems that merge via GW:  2508
Binary systems that evolve from a WR-BH and merge via GW:  2099


### Mass transfer events

In [8]:
df0_WRBH_stable = df0_WRBH.loc[df0_WRBH['BEvent']==5, ['ID','BEvent']].groupby('ID').count()

In [11]:
df0_WRBH_unstable = df0_WRBH.loc[(df0_WRBH['BEvent']==7) | (df0_WRBH['BEvent']==11), ['ID','BEvent']].groupby('ID').count()

In [16]:
mass_transfer0 = df0_WRBH_stable.merge(df0_WRBH_unstable, how='outer', on='ID').rename(columns={'BEvent_x': 'MTEvents_stable', 'BEvent_y': 'MTEvents_unstable'})

mass_transfer0['MTEvents_unstable'] = mass_transfer0['MTEvents_unstable'].fillna(0).astype(int)
mass_transfer0['MTEvents_stable'] = mass_transfer0['MTEvents_stable'].fillna(0).astype(int)

mass_transfer0.compute()


2024-05-09 12:37:37,649 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 672.73 MiB -- Worker memory limit: 0.93 GiB
2024-05-09 12:37:37,751 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 687.94 MiB -- Worker memory limit: 0.93 GiB
2024-05-09 12:37:37,868 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

,MTEvents_stable,MTEvents_unstable
ID,,
409,1,1
10348,1,0
20180,1,0
40366,1,0
40449,1,0
...,...,...
4160465,1,0
4160600,1,0
4170300,1,0
